(sec-modin)=
# Modin

Ray Data 提供的各类数据处理工具相对比较简单，只能做一些比较简单的转换，无法胜任复杂的数据处理任务。Modin 是一款专门加速 pandas 的框架。它对大数据进行了切分，使 DataFrame 分布到多核和集群上。早期，它底层使用了 Ray 作为分布式执行引擎，又被称为 Ray 上的 pandas（pandas on Ray）。之后又添加了 Dask 和 [unidist](https://github.com/modin-project/unidist/)，unidist 是 Modin 团队自己开发的分布式执行引擎。

在安装 Modin 时，要安装对应的执行引擎（Ray、Dask 或 unidist），比如 `pip install "modin[ray]"` 或 `pip install "modin[dask]"`。Modin 默认使用 Ray 作为其执行引擎。

## API 兼容性

Dask DataFrame 与 pandas DataFrame 其实有不少差异，很多 pandas 工作流并不能快速迁移到 Dask DataFrame 上。Modin 更看重与 pandas 的兼容性，用户只需要 `import modin.pandas as pd`，绝大多数 pandas 工作流可以快速迁移到 Modin 上。

Dask DataFrame 只按列对大数据进行切分，且没有记录每个 Partition 有多少数据，Modin 在多维度对数据进行切分，保留行标签和列标签。Modin 支持行索引 `iloc()`；记录了每个数据块的数据量，可以支持`median()`、`quantile()`；支持行和列的转换（比如，`pivot()`、`transpose()`）等操作。有关 Modin 的设计，可以参考其两篇论文 {cite}`petersohn2020Scalable` {cite}`petersohn2021Flexible`。

In [33]:
import os

import sys
sys.path.append("..")
from utils import nyc_flights

folder_path = nyc_flights()
file_path = os.path.join(folder_path, "*.csv")

:::{note}
Modin 的 API 尽量与 pandas 一致，比如，pandas 的 `read_csv()` 只能读一个文件，不能读 `*.csv` 这样的通配符。Modin 额外增加了一些 API，比如，Modin 拓展了 `read_csv()`，提出了 `read_csv_glob()` 方法 可以读取 `*.csv` 这样的通配符，适合读大数据。这些额外增加的 API 在 `modin.experimental.pandas` 中。
:::

In [34]:
import modin.experimental.pandas as pd
df = pd.read_csv_glob(file_path, parse_dates={'Date': [0, 1, 2]})
df.iloc[3]

Date                 1991-01-11 00:00:00
DayOfWeek                              5
DepTime                           1303.0
CRSDepTime                          1215
ArrTime                           1439.0
CRSArrTime                          1336
UniqueCarrier                         US
FlightNum                            121
TailNum                              NaN
ActualElapsedTime                   96.0
CRSElapsedTime                        81
AirTime                              NaN
ArrDelay                            63.0
DepDelay                            48.0
Origin                               EWR
Dest                                 PIT
Distance                           319.0
TaxiIn                               NaN
TaxiOut                              NaN
Cancelled                              0
Diverted                               0
Name: 3, dtype: object

In [35]:
df['ArrDelay'].median()

0.0

如果某些 API 在 Modin 中还没实现，Modin 会退回（Fallback）到 pandas，这也保证了兼容性。当然，缺点也很明显：将 Modin 的 DataFrame 转换为 pandas DataFrame 时，会有额外的开销；如果这个 DataFrame 分布在多个节点上，转化回 pandas 时会把数据集中到单机内存，有可能把单机内存挤爆。

## 立即执行

Modin 是立即执行，这一点与 pandas 一致。用户不需要像 Dask 那样调用 `.compute()` 来触发计算。

Modin 也没有 Dask DataFrame 的数据类型推断。{numref}`sec-dask-dataframe-read-write`中的飞机起降数据上，Dask DataFrame `tail()` 会抛出异常，但 Modin 能够得到 pandas 一样的语义。

In [36]:
df.tail(3)

,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,...,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
1555982,1994-12-27,2,1721.0,1715,1930.0,1945,DL,149,NaN,129.0,...,NaN,-15.0,6.0,JFK,ATL,760.0,NaN,NaN,0,0
1555983,1994-12-28,3,1715.0,1715,1934.0,1945,DL,149,NaN,139.0,...,NaN,-11.0,0.0,JFK,ATL,760.0,NaN,NaN,0,0
1555984,1994-12-29,4,1715.0,1715,1941.0,1945,DL,149,NaN,146.0,...,NaN,-4.0,0.0,JFK,ATL,760.0,NaN,NaN,0,0


## 执行引擎

Modin 支持 Ray、Dask 和 unidist 分布式执行引擎：可以利用单机多核，也可以运行在集群上。以 Ray 为例，用户可以向 Ray 集群上提交作业，在代码中初始 Ray 运行时 `ray.init(address="auto")` 后，会将作业运行 Ray 集群。

Modin 默认使用 Ray 作为执行后端，也可以通过环境变量 `MODIN_ENGINE` 来设置执行后端，在命令行里：`export MODIN_ENGINE=dask`；或在 Jupyter Notebook 中：

```python
import modin.config as modin_cfg
modin_cfg.Engine.put("ray")
```

undist 是 Modin 自己实现的一个执行后端，它支持 MPI，如果想用 undist MPI，除了设置 `MODIN_ENGINE` 还要设置 `UNIDIST_BACKEND`：

```shell
export MODIN_ENGINE=unidist
export UNIDIST_BACKEND=mpi 
```